In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm
import os
import re
import datetime

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from bs4 import BeautifulSoup

In [3]:
# 수집 날짜 지정(전일자)
before_one_day = datetime.datetime.now() - datetime.timedelta(days=1)
D, d = before_one_day.strftime("%Y%m%d000000"), before_one_day.strftime("%Y%m%d235959")
db_date = d[2:8]

In [9]:
# conn = db_connection('')
# news_link = pd.read_sql(f"select * db_name where text IS NULL;", conn)
# conn.close()

In [10]:
news_link = pd.read_xlsx(f'daum_news_{d}.xlsx')

print(len(news_link))
news_link.head()

468


,id,ck_idx,news,title,href,is_status,text,write_date,regist_date
0,656,1,뉴시스,이건희 3주기…이재용 회장 등 유족·사장단 추모 행렬(종합),http://v.daum.net/v/20231025153848989,2,None,None,2023-10-26 01:13:25
1,657,1,머니S,故 이건희 3주기 추도식… 이재용 회장 등 총수일가·삼성 사장단 참석,http://v.daum.net/v/20231025144046476,2,None,None,2023-10-26 01:13:25
2,658,1,연합뉴스,"오늘 故 이건희 선대회장 3주기…삼성, '신경영 정신' 되새긴다",http://v.daum.net/v/20231025050013307,2,None,None,2023-10-26 01:13:25
3,659,1,더팩트,새벽 귀국해 바로 선영으로…'이건희 3주기' 기린 이재용·삼성 사장단(영상),http://v.daum.net/v/20231025125112595,2,None,None,2023-10-26 01:13:25
4,660,1,디지털타임스,"이재용 ""아버지 뵈러 왔습니다""… 故 이건희 추도식 참석",http://v.daum.net/v/20231025143332063,2,None,None,2023-10-26 01:13:25


---

In [11]:
# 뉴스 내용 담을 DataFrame
news_text = pd.DataFrame(columns = ['title', 'text', 'write_date'])

drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in tqdm(range(len(news_link))):
    drv.get(news_link['href'][i])
    time.sleep(1.5)
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    text_list = soup.find('div', {'class':'article_view'}).findAll('p')
    text = ''
    for k in range(len(text_list)):
        text += text_list[k].get_text()
        text += ' '
    text = text.strip().replace('\xa0', '').replace("\'", '').replace('"', '')
    
    if text == '':
        text = soup.find('div', {'class':'article_view'}).find('div').get_text()
        text = text.replace('\n', '').replace('          ', "").replace('\xa0', '').replace("\'", '').replace('"', '').strip()
    
    date_list = soup.find('div', {'class':'info_view'}).get_text().split('수정')[0].split('입력')[-1].strip().split('. ')#.replace('. ', '-')#.split(' ')[0][:-1].replace('.', '-')
    date = ''
    for j in range(len(date_list[:-1])):
        date += date_list[j]
        date += '-'

    data = pd.DataFrame({'id':news_link['id'][i],'title':news_link['title'][i], 'write_date':[date[:-1]], 'text':[text]})
    news_text = pd.concat([news_text, data], ignore_index=True)

drv.close()

100%|█████████████████████████████████████████| 200/200 [08:19<00:00,  2.50s/it]


In [12]:
news_text

,title,text,write_date,id
0,이건희 3주기…이재용 회장 등 유족·사장단 추모 행렬(종합),[수원=뉴시스]이인준 기자 = 고(故) 이건희 삼성 회장의 3주기 추모식이 25일 ...,2023-10-25,656.0
1,故 이건희 3주기 추도식… 이재용 회장 등 총수일가·삼성 사장단 참석,"이날 추도식에는 고인의 아내인 홍라희 전 삼성미술관 리움 관장, 장남 이재용 삼성전...",2023-10-25,657.0
2,"오늘 故 이건희 선대회장 3주기…삼성, '신경영 정신' 되새긴다",(서울=연합뉴스) 장하나 기자 = 고(故) 이건희 삼성 선대회장이 별세한 지 25일...,2023-10-25,658.0
3,새벽 귀국해 바로 선영으로…'이건희 3주기' 기린 이재용·삼성 사장단(영상),"이재용·홍라희·이부진·이서현 등 삼성가, 수원 선영서 조용한 추도식 이건희 선대회...",2023-10-25,659.0
4,"이재용 ""아버지 뵈러 왔습니다""… 故 이건희 추도식 참석",고(故) 이건희 삼성 선대회장의 3주기 추도식이 25일 경기도 수원 선영에서 진행됐...,2023-10-25,660.0
...,...,...,...,...
195,"""주변 승마선수 아무도 몰라"" '최순실 딸' 정유라, '남현희♥' 전청조 맹비난",[스포츠조선 백지은 기자] 최서연(개명 전 최순실) 딸 정유라가 펜싱 국가대표 출신...,2023-10-25,1907.0
196,"선수가 아니라 감독이 '지구 라이벌'로? 멜빈, SF 사령탑으로 이적",충격에 가까운 소식이다. 밥 멜빈 샌디에이고 파드리스 감독이 지구 라이벌 샌프란시스...,2023-10-25,1908.0
197,2024년 K리그부터 '원정 응원석 차별' 사라진다…1부는 출전 선수 20명 확대 ...,(엑스포츠뉴스 김현기 기자)K리그 관전의 숨은 걸림돌로 꼽히는 원정 응원석 차별 금...,2023-10-25,1909.0
198,선수 독려하는 NC 강인권 감독,(창원=뉴스1) 윤일지 기자 = 25일 오후 경남 창원NC파크에서 열린 프로야구 2...,2023-10-25,1910.0


In [ ]:
# 저장(덮어쓰기)
news_text.to_xlsx(f'daum_news_{d}.xlsx')

In [13]:
# for i in tqdm(range(len(news_text))):
#     conn = db_connection('')
#     cur = conn.cursor()

#     cur.execute(
#         f'update db_name set text="{news_text["text"][i]}", write_date="{news_text["write_date"][i]}" where id = "{news_text["id"][i]}"'
#     )
#     conn.commit()

#     cur.close()
#     conn.close()

100%|█████████████████████████████████████████| 200/200 [01:04<00:00,  3.10it/s]
